# Общая информация:
__Задачи, решаемые в ноутбуке:__

1) Сделать эвалуацию базовой GPT neo на случайно из data part 4, включающей 105 и 500 экземпляров (API Usage category, <=200 length)

2) Отобрать экземпляров, имеющих наименьшее косинусное расстояние Q_title и Q_Body

3) Сделать эвалуацию базовой модели на отобранной выборке

4) Сравнить результат

5) Сделать кластрезицию на основе BERToflow embedding

# Импорт библиотек

In [11]:
if 'google.colab' in str(get_ipython()):
    !pip install bert_score datasets rouge_score evaluate pynvml transformers wandb sentence-transformers
    
    !wget https://raw.githubusercontent.com/Myashka/Diploma/master/Notebooks/utils.py

    from utils import *

    from google.colab import drive
    drive.mount("/content/drive")
else:
    import sys
    sys.path.append(r"D:\vkr\Notebooks")

    from utils import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-11-16 23:25:11--  https://raw.githubusercontent.com/Myashka/Diploma/master/Notebooks/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8246 (8.1K) [text/plain]
Saving to: ‘utils.py.2’

utils.py.2          100%[===================>]   8.05K  --.-KB/s    in 0s      

2022-11-16 23:25:11 (67.1 MB/s) - ‘utils.py.2’ saved [8246/8246]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_colwidth", None)

from tqdm.auto import tqdm
import torch
from transformers import (AutoModelForTokenClassification, AutoTokenizer, AutoModelForCausalLM)
from sentence_transformers import SentenceTransformer

In [2]:
torch.manual_seed(42)
np.random.seed(42)

# Чтение данных

In [5]:
df = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/vkr_data/df_200.csv')
df = df.drop(columns=['Unnamed: 0'])
df = df.reset_index(drop=True)

In [6]:
df = df.sort_values('t_q_BERT_dist', ascending=False)
#df = df.sort_values('t_q_MPnet_dist', ascending=False)

In [7]:
samples = 105

In [ ]:
eval_df = df.sample(samples)
eval_df = eval_df.reset_index(drop=True)

In [8]:
# GOOD samples
eval_df = df.head(samples)

In [ ]:
# BAD samples
eval_df = df.tail(samples)

In [3]:
device = torch.device("cuda") if torch.cuda.is_available else "cpu"

# SO MPnet embeddings

In [ ]:
def get_embeddings(df, model, tokenizer, column, device):
    model.to(device)
    batch_size = 100
    rows = df[f'{column}']
    max_length = max([len(tokenizer.encode(row)) for row in rows])
    if max_length > 512:
        max_length = 512
      
    embeddings = torch.empty((len(df), 768)).to(device)

    with torch.no_grad():
        for i in tqdm(range(0, len(embeddings), batch_size)):
            tokenized_batch = tokenizer.batch_encode_plus(rows[i: i+batch_size].tolist(), add_special_tokens=True, padding='max_length', max_length=max_length, truncation=True, return_tensors="pt")

            tokenized_batch = tokenized_batch.to(device)
    
            last_hidden_states = model(**tokenized_batch)

            emb_batch = last_hidden_states[0][:,0,:]

            embeddings[i: i+batch_size] = emb_batch
    return embeddings.detach().cpu()

In [ ]:
mp_tokenizer = AutoTokenizer.from_pretrained("flax-sentence-embeddings/stackoverflow_mpnet-base")
mp_model = AutoModelForTokenClassification.from_pretrained("flax-sentence-embeddings/stackoverflow_mpnet-base")
mp_model = mp_model.mpnet

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of MPNetForTokenClassification were not initialized from the model checkpoint at flax-sentence-embeddings/stackoverflow_mpnet-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
q_emb_mp = get_embeddings(df, mp_model, mp_tokenizer, 'Q_Body', device)

  0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
title_emb_mp = get_embeddings(df, mp_model, mp_tokenizer, 'Q_Title', device)

  0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
mp_cos_dist = torch.nn.functional.cosine_similarity(q_emb_mp, title_emb_mp, dim=1)

In [ ]:
df['t_q_MPnet_dist'] = pd.Series(list(mp_cos_dist), index=df.index)

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/vkr_data/df_200.csv')

# BERToflow embeddings

In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained("lanwuwei/BERTOverflow_stackoverflow_github")
bert_model = AutoModelForTokenClassification.from_pretrained("lanwuwei/BERTOverflow_stackoverflow_github")
bert_model = bert_model.bert

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.24k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/660k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/596M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at lanwuwei/BERTOverflow_stackoverflow_github and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
bert_question_emb = get_embeddings(df, bert_model, bert_tokenizer, 'Q_Body', device)

  0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
bert_title_emb = get_bert_embeddings(df, bert_model, bert_tokenizer, 'Q_Title', device)

  0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
bert_cos_dist = torch.nn.functional.cosine_similarity(bert_question_emb, bert_title_emb, dim=1)

In [ ]:
df['t_q_BERT_dist'] = pd.Series(list(bert_cos_dist), index=df.index)

In [ ]:
df = df.sort_values('t_q_BERT_dist', ascending=False)

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/vkr_data/df_200.csv')

## Cosine similarity между BERT И MPnet embeddings

In [ ]:
df

,Id,Q_date_open,Q_Score,Q_Title,Q_Body,A_Score,A_Body,Tag,Q_len,A_len,API_CHANGE,API_USAGE,CONCEPTUAL,DISCREPANCY,DOCUMENTATION,ERRORS,REVIEW,t_q_BERT_dist,t_q_MPnet_dist
695,11947640,2012-08-14 07:27:37+00:00,16,"how to use ""Select debug app"" and ""wait for debugger"" new feature in jelly bean?","all""Select debug app"" and ""wait for debugger"" are new feature in Jelly Bean, does someone know how to use these new feature? what do they do?Thanks",2.0,"It is a mystery to me also, but I'll share my experience. I cannot see that these options change the phone's behavior. Regardless of the settings I choose, the Galaxy Nexus behaves like my older phones that do not have the settings.""Waiting for debugger"" is a dialog that I see whenever I start running an app from Eclipse. It stays up for a while, and then my app starts running. This has been the case since I started Android development, and it still works that way on my old and new devices, regardless of the setting.If I click on ""Select Debug App"", it allows me to select an app from the already installed apps on my phone that allow debugging, which are the apps that I wrote myself.On my Galaxy Nexus it does not matter whether I select ""Nothing"" or one of my debuggable apps, I can still debug any app from Eclipse. I cannot even see a difference in time it takes to load an app over the ADB connection.","['android', 'android-4.2-jelly-bean']",28,179,0,1,0,0,0,0,0,tensor(0.9910),tensor(0.8872)
692,4397360,2010-12-09 10:46:15+00:00,0,android: how to access from activity A a non static public function of activity B at runtime?,Does anyone know in android how you can call a non static public function of an activity from another activity ?example : activity A want to access public function toto of activity Bthanks,1.0,"case 1: if you are executing yours activitys into an activitygroup, then you can obtain the interested activity instance from LocalActivityManager doing something like this YouractivitygetLocalactivityManager.getActivityyourActivityId.performYourMethod...Case 2: if you aren't running your activitys into an activitygroup, then you can use the activity B method through broadcast, is other way to perform inter activity communication.Cheer","['android', 'function', 'android-activity']",33,59,0,1,0,0,0,0,0,tensor(0.9900),tensor(0.7121)
651,17446460,2013-07-03 11:12:26+00:00,0,get how much time i have before the battery goes to 0% Android,Can i get how much time i have before the battery goes to 0%? Something using a CountDownTimer and making a stime of how much mAh the battery consume? Someone can help me to do it?,1.0,"Short answer: No.Long answer: Yes, you can.. But not that easily.. Because you don't know how much energy does the device consume at a time basis, you should first try to find that out and then calculate the remaining time using that data... As you might have guessed already, it might not be that accurate at first but the more you sample, the more accurate that data will be..So to conclude here is what you have to do:First when the user opens the app, because you don't have enough data about energy consumption, You should just display something like Calculating.... Here you should decide to have a sample time. That is something like derivation in math.. Let's say you have a sample time of 10 seconds. Your actual one should be bigger I guess. You should have a variable named something like level1.. Save the current level to it. Wait for the amount of sample time which is now 10 seconds.. Now save the current value to level2.. Save the level1 - level2 product to another variable like consumed.Now here is the magic. You know how much power has been consumed in 10 seconds That is the consumed variable.. From now on you can calculate the remaining time using that data. But remember it's still one 10 second. What if that 10 seconds user was playing a heavy game or even the device was on stand by? That is why it's not accurate. But until now.. What you can do

In [ ]:
str_1 = 'What is the weather like'
str_2 = 'What is the weather like'

tok_1 = bert_tokenizer(str_1, add_special_tokens=True, padding='max_length', max_length=512, truncation=True, return_tensors="pt")
tok_2 = bert_tokenizer(str_2, add_special_tokens=True, padding='max_length', max_length=512, truncation=True, return_tensors="pt")

tok_1 = tok_1.to(device)
tok_2 = tok_2.to(device)

emb_1 = bert_model(**tok_1)[0][:,0,:]
emb_2 = bert_model(**tok_2)[0][:,0,:]

In [ ]:
torch.nn.functional.cosine_similarity(emb_1, emb_2, dim=1)

tensor([1.0000], device='cuda:0', grad_fn=<SumBackward1>)

# Загрузка модели

In [4]:
model_name = "EleutherAI/gpt-neo-1.3B"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [12]:
promt_1 = """You will be asked a question. For this question, you must answer by its title and question body.\nTitle: """
#promt_1 = ''
promt_2 = '\nQuestion body: '
#promt_2 = ''
promt_3 = '\nAnswer:'
promts = [promt_1, promt_2, promt_3]

use_title = True
use_question = True

In [13]:
dataset = Q_A_Dataset(eval_df, tokenizer, *promts, use_title=use_title, use_question=use_question)

# Инициализация в WandB

In [14]:
wandb.login()
run = wandb.init(project="QA specific domain", entity="myashka")

%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all
%env WANDB_SILENT=true

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: myashka. Use `wandb login --relogin` to force relogin


env: WANDB_LOG_MODEL=true
env: WANDB_WATCH=all
env: WANDB_SILENT=true


In [15]:
run_table_name = 'base_android_api_usage_question_answer_samples'
run_table = wandb.Artifact(f"{run_table_name}", type="run_table")

# Запуск evaluation

In [16]:
evaluator = Evaluator(run_table, model, tokenizer)

In [ ]:
eval_table, bleu, rouge, bert_pr, bert_rec, bert_f1 = evaluator.evaluate(dataset, *promts, use_title=use_title, use_question=use_question)

  0%|          | 0/105 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

In [ ]:
eval_filename = 'base_bert_good_105_samples_title_question'

In [ ]:
run_table.add(eval_table, f"{eval_filename}")
run.log_artifact(run_table)
wandb.finish()

In [ ]:
print(bleu, rouge, bert_pr, bert_rec, bert_f1)

0.0807711943967461 0.13015668877684844 0.8122783655212039 0.8324493374143328 0.8214696532204038


In [ ]:
print(bleu, rouge, bert_pr, bert_rec, bert_f1)

0.07735358066254513 0.11894398292346023 0.8047025003433228 0.8320511833429337 0.8173794143199921


In [ ]:
print(bleu, rouge, bert_pr, bert_rec, bert_f1)

0.07419551054249134 0.13119578324627432 0.8018379024096898 0.8336302467754909 0.8167109506470817
